In [1]:
import time
time_start=time.time()

In [2]:
!pip install -U gdown
import gdown

file_id = "1YFs_NLzPMqAzZekZYHxykG_qp2vg6qdP"
gdown.download(f"https://drive.google.com/uc?id={file_id}", output="data.zip", quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1YFs_NLzPMqAzZekZYHxykG_qp2vg6qdP
From (redirected): https://drive.google.com/uc?id=1YFs_NLzPMqAzZekZYHxykG_qp2vg6qdP&confirm=t&uuid=bf3bbb5f-abc3-40bf-a11f-899e81432f53
To: /content/data.zip
100%|██████████| 122M/122M [00:02<00:00, 59.3MB/s]


'data.zip'

In [3]:
import zipfile
with zipfile.ZipFile("data.zip", 'r') as zip_ref:
    zip_ref.extractall("data")

In [4]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
import json
import xml.etree.ElementTree as ET
import glob

import torch
import torch.nn.functional as F
from torch import nn
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import get_cosine_schedule_with_warmup
from torch.utils.data import Dataset

import cv2, glob, json, math, os, random, xml.etree.ElementTree as ET
from pathlib import Path
import numpy as np

from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [5]:
data_root="/content/data"

img_size=256

# Model

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm

class UniHeadNet(nn.Module):
    def __init__(self,
                 n_det_cls=10, n_seg_cls=21, n_cls=30,
                 n_anchor=3):
        super().__init__()
        self.backbone = timm.create_model(
            'efficientnet_b0',
            pretrained=True,
            features_only=True,
            out_indices=(3,)  # block6 output: (B, 320, 16, 16) for 256x256 input
        )
        #3 112 2 40
        self.reduce = nn.Sequential(
            nn.Conv2d(112, 256, kernel_size=1, bias=False),
            nn.BatchNorm2d(256),
            nn.SiLU()
        )
#         self.upsample = nn.Upsample(scale_factor=2, mode='bilinear',
#                                     align_corners=False)

        self.head_conv = nn.Sequential(
            nn.Conv2d(256, 256, 3, padding=1, bias=False),
            nn.BatchNorm2d(256), nn.SiLU())

        C_det = n_anchor * (5 + n_det_cls)
        C_seg = n_seg_cls
        C_cls = n_cls
        self.C_det, self.C_seg, self.C_cls = C_det, C_seg, C_cls

        self.head_out = nn.Conv2d(256, C_det + C_seg + C_cls, 1)

    def forward(self, x, export_onnx=False):
#         m = self.backbone
#         feat = m.forward_features(x)             # (B,1280,7,7)
        feat = self.backbone(x)[0]
        #print(feat.shape)
        #print(feat.shape)
        #feat = self.upsample(self.reduce(feat))
        feat = self.reduce(feat) # (B,256,14,14)
        #print(feat.shape)
        y = self.head_out(self.head_conv(feat))  # (B, C_all, 14, 14)

        #-------------- 拆解三種輸出 ----------------#
        B = y.size(0)
        C_det, C_seg, C_cls = self.C_det, self.C_seg, self.C_cls

        det_logits = y[:, :C_det]                    # (B,C_det,14,14)

        seg_logits = y[:, C_det:C_det + C_seg]       # (B,C_seg,14,14)
        seg_logits = F.interpolate(seg_logits, size=x.shape[2:],
                                   mode='bilinear', align_corners=False)

        cls_logits = y[:, -C_cls:]                  # (B,C_cls,14,14)
        cls_logits = cls_logits.mean(dim=[2, 3])     # GAP → (B,C_cls)

        if export_onnx:      # 方便部署
            return det_logits, seg_logits, cls_logits
        return {
            'det': det_logits,      # 後處理：decode boxes + NMS
            'seg': seg_logits,      # CrossEntropy2d
            'cls': cls_logits       # CE / focal loss
        }


In [7]:
dummy = torch.randn(1, 3, img_size, img_size)
model = UniHeadNet(
    n_det_cls=10,   # 偵測 10 個類別
    n_seg_cls=19,   # 分割 19 個類別（含背景）
    n_cls=30,       # 影像分類 10 個類別
    n_anchor=3      # 每個網格 3 個 anchor
).eval()
with torch.no_grad():
    out = model(dummy)   # forward

for k, v in out.items():
    print(f"{k:3} ➜ {tuple(v.shape)}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

det ➜ (1, 45, 16, 16)
seg ➜ (1, 19, 256, 256)
cls ➜ (1, 30)


In [8]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params / 1e6:.2f} M")


Total parameters: 4.24 M


# Seg

In [9]:
def build_class_dict(xml_dir):
    cls_set = set()
    for p in glob.glob(f'{xml_dir}/*.xml'):
        root = ET.parse(p).getroot()
        name = root.find('.//object/name').text.strip()
        cls_set.add(name)
    cls2id = {c:i+1 for i,c in enumerate(sorted(cls_set))}
    return cls2id

cls2id=build_class_dict(f"{data_root}/data/mini_voc_seg/train/Annotations")


In [10]:
import xml.etree.ElementTree as ET
import cv2, torch
from torch.utils.data import Dataset
import numpy as np

class SegMiniN(Dataset):
    def __init__(self, names, root, cls2id, size=256, lb=True):
        """
        names : list[str]   # 不含副檔名
        root  : Path        # 資料根目錄，下有 JPEGImages / SegmentationClass / Annotations
        """
        self.names, self.root = names, Path(root)
        self.size, self.lb = size, lb
        self.cls2id = cls2id    # 已事先建立的 {class: id}

    def _letterbox(self, img, mask):
        h, w = img.shape[:2]
        s = self.size / max(h, w)
        nh, nw = int(h * s), int(w * s)
        img_r = cv2.resize(img, (nw, nh))
        msk_r = cv2.resize(mask, (nw, nh), interpolation=cv2.INTER_NEAREST)

        out_i = np.zeros((self.size, self.size, 3), np.uint8)
        out_m = np.zeros((self.size, self.size),   np.uint8)
        top, left = (self.size - nh)//2, (self.size - nw)//2
        out_i[top:top+nh, left:left+nw] = img_r
        out_m[top:top+nh, left:left+nw] = msk_r
        return out_i, out_m

    def __getitem__(self, idx):
        name = self.names[idx]

        img = cv2.imread(f"{self.root}/JPEGImages/{name}.jpg")[:, :, ::-1]  # BGR→RGB

        # 讀 mask；如是彩色 palette，這裡簡化：只判斷 >0 為前景
        mask_png = cv2.imread(f"{self.root}/SegmentationClass/{name}.png",
                              cv2.IMREAD_UNCHANGED)


        target_color = np.array([192, 224, 224], dtype=np.uint8)

        # 建立布林遮罩：True 表示「這個像素就是目標顏色」
        mask = (mask_png  == target_color).all(axis=-1)   # shape = (512, 512)

        # 把符合條件的像素設成 [0,0,0]
        mask_png[mask] = [0, 0, 0]


        if mask_png.ndim == 3:

            mask_png = mask_png.sum(axis=2)

        # 讀 XML → 取 <name>
        xml_path = f"{self.root}/Annotations/{name}.xml"
        cls_name = ET.parse(xml_path).find('.//object/name').text.strip()
        fg_id = self.cls2id[cls_name]
        #print(fg_id)
        mask = np.where(mask_png > 0, fg_id, 0).astype(np.uint8)

        # 尺寸處理
        if self.lb:
            img, mask = self._letterbox(img, mask)
        else:
            img = cv2.resize(img, (self.size, self.size))
            mask = cv2.resize(mask, (self.size, self.size), interpolation=cv2.INTER_NEAREST)

        # to Tensor
        img = torch.from_numpy(img).permute(2,0,1).float() / 255.
        mask = torch.from_numpy(mask).long()        # 先轉成 int64
        mask_1hot = torch.nn.functional.one_hot(
            mask, num_classes=len(self.cls2id)+1      # (H,W,k)
        ).permute(2, 0, 1).float()

        return img, mask_1hot         # img:(3,256,256)  mask:(k,256,256)

    def __len__(self): return len(self.names)


In [11]:


# ──────────────────── 0. 基本參數 ──────────────────── #
CFG = dict(
    root_tr      = Path(f"{data_root}/data/mini_voc_seg/train"),
    root_val     = Path(f"{data_root}/data/mini_voc_seg/val"),
    img_size     = img_size,
    batch_size   = 8,
    epochs       = 200,
    lr           = 1e-5,
    seed         = 42,
    device       = "cuda" if torch.cuda.is_available() else "cpu",
    num_workers  = 2,
    save_dir     = Path("ckpt_stage1"),
)

cfg=CFG
tr_names = sorted(p.stem for p in (cfg["root_tr"] / "Annotations").glob("*.xml"))
val_names= sorted(p.stem for p in (cfg["root_val"] / "Annotations").glob("*.xml"))
train_ds = SegMiniN(tr_names, cfg["root_tr"], cls2id, cfg["img_size"])
val_ds   = SegMiniN(val_names,  cfg["root_val"], cls2id, cfg["img_size"])

In [12]:


def dice_loss(pred, target, epsilon=1e-6):
    """
    pred: (B, C, H, W) after sigmoid
    target: (B, C, H, W) one-hot
    """
    pred = torch.sigmoid(pred)  # 先做 sigmoid
    intersect = (pred * target).sum(dim=(2, 3))
    union = pred.sum(dim=(2, 3)) + target.sum(dim=(2, 3))
    dice = (2 * intersect + epsilon) / (union + epsilon)
    return 1 - dice.mean()

def seg_loss_fn(pred_logits, target_1hot):
    """
    pred_logits: raw logits, (B, C, H, W)
    target_1hot: one-hot, (B, C, H, W)
    """
    bce = F.binary_cross_entropy_with_logits(pred_logits, target_1hot)
    dice = dice_loss(pred_logits, target_1hot)
    return 0.5 * bce + 0.5 * dice


def train_seg(model, train_loader, optimizer, device):
    model.train()
    total_loss = 0

    for img, mask_1hot in tqdm(train_loader):
        img = img.to(device)
        mask_1hot = mask_1hot.to(device)

        out = model(img)['seg']  # get segmentation output

        loss = seg_loss_fn(out, mask_1hot)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Train Loss: {avg_loss:.4f}")
    return avg_loss

@torch.no_grad()
def validate_seg(model, val_loader, device):
    model.eval()
    total_loss = 0
    total_mIoU = 0
    num_classes = val_loader.dataset[0][1].shape[0]  # (C, H, W)

    for img, mask_1hot in tqdm(val_loader):
        img = img.to(device)
        mask_1hot = mask_1hot.to(device)

        out = model(img)['seg']

        loss = seg_loss_fn(out, mask_1hot)
        total_loss += loss.item()

        # mIoU
        pred = torch.sigmoid(out) > 0.5
        iou = compute_mIoU(pred, mask_1hot)
        total_mIoU += iou.item()

    avg_loss = total_loss / len(val_loader)
    avg_mIoU = total_mIoU / len(val_loader)
    print(f"Val Loss: {avg_loss:.4f}, mIoU: {avg_mIoU:.4f}")
    return avg_loss, avg_mIoU
def compute_mIoU(pred, target, epsilon=1e-6):
    """
    pred: (B, C, H, W) binary after sigmoid > 0.5
    target: (B, C, H, W)
    """
    intersection = (pred & target.bool()).sum(dim=(2, 3))
    union = (pred | target.bool()).sum(dim=(2, 3))
    iou = (intersection + epsilon) / (union + epsilon)
    return iou.mean()

In [13]:


train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=2)
val_seg_loader = DataLoader(val_ds, batch_size=8)

model = UniHeadNet(n_seg_cls=19,n_det_cls=10, n_anchor=3,n_cls=30).to(cfg["device"])
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
best_miou=0
for epoch in range(1, 21):
    print(f"\nEpoch {epoch}")
    train_seg(model, train_loader, optimizer, cfg["device"])
    _,val_miou=validate_seg(model, val_seg_loader, cfg["device"])

    if val_miou > best_miou:
      best_miou = val_miou
      torch.save(model.state_dict(), 'best_seg_model.pth')
      print(f"Best mIOU is {best_miou}")
      print("save model")


Epoch 1


100%|██████████| 30/30 [00:12<00:00,  2.45it/s]


Train Loss: 0.7577


100%|██████████| 8/8 [00:04<00:00,  1.66it/s]


Val Loss: 0.6986, mIoU: 0.2841
Best mIOU is 0.28414864279329777
save model

Epoch 2


...

Val Loss: 0.4800, mIoU: 0.9059

Epoch 20


100%|██████████| 30/30 [00:06<00:00,  4.89it/s]


Train Loss: 0.4615


100%|██████████| 8/8 [00:01<00:00,  4.29it/s]


Val Loss: 0.4805, mIoU: 0.9074
Best mIOU is 0.9073649048805237
save model


In [14]:
model.load_state_dict(torch.load('best_seg_model.pth'))
_,val_best_miou = validate_seg(model, val_seg_loader, cfg["device"])
print(f"mIoU_base is:{val_best_miou}")

100%|██████████| 8/8 [00:02<00:00,  3.86it/s]

Val Loss: 0.4805, mIoU: 0.9074
mIoU_base is:0.9073649048805237


# Det

In [15]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import os, json
import random
import torchvision.transforms as T
import torchvision.transforms.functional as TF

class COCODetectionDataset(Dataset):
    def __init__(self, img_dir, ann_path, det_cat_mapping, size=256):
        self.img_dir = img_dir
        self.ann_path = ann_path
        self.size = size
        self.det_cat_mapping = det_cat_mapping

        with open(ann_path, 'r') as f:
            coco = json.load(f)

        self.id2file = {img['id']: img['file_name'] for img in coco['images']}

        self.imgid2anns = {}
        for ann in coco['annotations']:
            if ann['iscrowd']:
                continue
            img_id = ann['image_id']
            if img_id not in self.imgid2anns:
                self.imgid2anns[img_id] = []
            self.imgid2anns[img_id].append(ann)

        self.image_ids = [k for k, v in self.imgid2anns.items() if len(v) > 0]

        # 圖像增強設定
        self.color_jitter = T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3)

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        file_name = self.id2file[img_id]
        img_path = os.path.join(self.img_dir, file_name)

        img = Image.open(img_path).convert("RGB")
        orig_w, orig_h = img.size

        # 只取第一個 bbox
        ann = self.imgid2anns[img_id][0]
        x, y, w, h = ann['bbox']
        x1, y1, x2, y2 = x, y, x + w, y + h

        # 隨機水平翻轉
        if random.random() < 0.5:
            img = TF.hflip(img)
            x1, x2 = orig_w - x2, orig_w - x1  # 對 x1,x2 做鏡像

        # 隨機色彩抖動
        if random.random() < 0.5:
            img = self.color_jitter(img)

        # Resize 並同步 bbox 縮放
        img = TF.resize(img, (self.size, self.size))
        scale_x = self.size / orig_w
        scale_y = self.size / orig_h
        x1 *= scale_x
        x2 *= scale_x
        y1 *= scale_y
        y2 *= scale_y

        box = torch.tensor([[x1, y1, x2, y2]], dtype=torch.float32)

        original_id = ann['category_id']
        label = self.det_cat_mapping.get(original_id, -1)
        if label == -1:
            raise ValueError(f"Category ID {original_id} not in det_cat_mapping")
        label = torch.tensor([label], dtype=torch.int64)

        img_tensor = TF.to_tensor(img)

        target = {
            "boxes": box,             # (1, 4)
            "labels": label,          # (1,)
            "image_id": torch.tensor([img_id])
        }

        return img_tensor, target



In [16]:
import json

# 開啟並讀取 json 檔
with open(f'{data_root}/data/mini_coco_det/annotations/instances_train2017.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 現在 data 是一個 Python 字典 (dict) 或列表 (list)


cat_id=[]
for i in range(len(data["annotations"])):
    cat_id.append(data["annotations"][i]["category_id"])

det_cat_mapping = {item: idx for idx, item in enumerate(set(cat_id))}
det_cat_mapping

{5: 0, 70: 1, 7: 2, 11: 3, 13: 4, 16: 5, 85: 6, 23: 7, 24: 8, 25: 9}

In [17]:
rev_det_cat_mapping = {v: k for k, v in det_cat_mapping.items()}

In [18]:
def collate_fn(batch):
    imgs, targets = list(zip(*batch))
    imgs = torch.stack(imgs, dim=0)
    return imgs, list(targets)


def decode_boxes(det_logits, anchors, stride=16):
    """
    det_logits: (B, A*(5+C), H, W)
    anchors: (A, 2) anchor 尺寸
    return: decoded boxes in (B, A*H*W, 4), confs, class_probs
    """
    B, _, H, W = det_logits.shape
    A = anchors.shape[0]
    C = det_logits.shape[1] // A - 5  # 推出 class 數

    pred = det_logits.view(B, A, 5 + C, H, W).permute(0, 1, 3, 4, 2)  # (B,A,H,W,5+C)
    tx, ty, tw, th, obj = pred[..., 0], pred[..., 1], pred[..., 2], pred[..., 3], pred[..., 4]
    cls_logits = pred[..., 5:]

    grid_y, grid_x = torch.meshgrid(torch.arange(H), torch.arange(W), indexing='ij')
    grid = torch.stack((grid_x, grid_y), dim=-1).to(det_logits.device)  # (H,W,2)

    # sigmoid + decode
    cx = (grid[..., 0][None, None] + tx.sigmoid()) * stride
    cy = (grid[..., 1][None, None] + ty.sigmoid()) * stride
    w = (tw.exp() * anchors[:, 0][:, None, None]).to(det_logits.device)
    h = (th.exp() * anchors[:, 1][:, None, None]).to(det_logits.device)

    x1 = cx - w / 2
    y1 = cy - h / 2
    x2 = cx + w / 2
    y2 = cy + h / 2

    boxes = torch.stack([x1, y1, x2, y2], dim=-1)  # (B,A,H,W,4)
    boxes = boxes.reshape(B, -1, 4)
    confs = obj.sigmoid().reshape(B, -1)
    class_probs = cls_logits.softmax(-1).reshape(B, -1, C)

    return boxes, confs, class_probs


def yolo_loss(det_logits, targets, anchors, num_classes,
              img_size=img_size, lambda_coord=5.0, lambda_noobj=0.5):
    B, _, H, W = det_logits.shape
    A = anchors.shape[0]
    device = det_logits.device
    stride = img_size / H

    pred = det_logits.view(B, A, 5 + num_classes, H, W).permute(0, 1, 3, 4, 2)

    total_loss, bbox_loss, obj_loss, cls_loss = 0.0, 0.0, 0.0, 0.0

    for i in range(B):
        target = targets[i]
        gt_box = target['boxes'][0]
        gt_label = target['labels'][0]
        anchors = anchors.to(gt_box.device)

        cx = (gt_box[0] + gt_box[2]) / 2 / stride
        cy = (gt_box[1] + gt_box[3]) / 2 / stride
        w = (gt_box[2] - gt_box[0]) / anchors[:, 0]
        h = (gt_box[3] - gt_box[1]) / anchors[:, 1]

        gi, gj = int(cx), int(cy)

        for a in range(A):
            pred_a = pred[i, a, gj, gi]
            pred_tx, pred_ty, pred_tw, pred_th, pred_obj = pred_a[:5]
            pred_cls = pred_a[5:]

            # 坐標損失 (tx, ty 是 sigmoid, tw, th 是 log-scale regression)
            bbox_loss += lambda_coord * F.mse_loss(pred_tx.sigmoid(), cx - gi)
            bbox_loss += lambda_coord * F.mse_loss(pred_ty.sigmoid(), cy - gj)
            bbox_loss += lambda_coord * F.mse_loss(pred_tw, torch.log(w[a]))
            bbox_loss += lambda_coord * F.mse_loss(pred_th, torch.log(h[a]))

            # 物件性損失
            obj_loss += F.binary_cross_entropy_with_logits(pred_obj, torch.tensor(1.0, device=device))

            # 分類損失（使用 softmax + cross entropy）
            cls_loss += F.cross_entropy(pred_cls.unsqueeze(0), gt_label.unsqueeze(0))

    total_loss = bbox_loss + obj_loss + cls_loss
    return total_loss / B, bbox_loss / B, obj_loss / B, cls_loss / B

def train_one_epoch(model, dataloader, optimizer, anchors, num_classes, device):
    model.train()
    total_loss = 0.0

    for imgs, targets in tqdm(dataloader):
        imgs = imgs.to(device)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        outputs = model(imgs)
        det_logits = outputs['det']

        loss, loss_bbox, loss_obj, loss_cls = yolo_loss(det_logits, targets, anchors, n_det_cls)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #print(f"Loss: {loss:.3f}, BBox: {loss_bbox:.3f}, Obj: {loss_obj:.3f}, Cls: {loss_cls:.3f}")

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

def compute_iou(box1, box2):
    """
    box1, box2: (4,) → [x1, y1, x2, y2]
    """
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    inter_area = max(x2 - x1, 0) * max(y2 - y1, 0)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = area1 + area2 - inter_area

    return inter_area / union_area if union_area > 0 else 0
import torch
from torchvision.ops import nms
from tqdm import tqdm

@torch.no_grad()
def evaluate_map(model, val_loader, anchors, num_classes, device, iou_thresh=0.5, conf_thresh=0.5):
    model.eval()
    total = 0
    correct = 0

    for imgs, targets in tqdm(val_loader):
        imgs = imgs.to(device)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        outputs = model(imgs)
        det_logits = outputs['det']  # (B, A*(5+C), H, W)

        boxes, confs, class_probs = decode_boxes(det_logits, anchors.to(device))
        pred_scores = confs * class_probs.max(dim=-1).values  # (B, N)
        pred_labels = class_probs.argmax(dim=-1)              # (B, N)

        for i in range(len(imgs)):
            score = pred_scores[i]   # (N,)
            label = pred_labels[i]   # (N,)
            box = boxes[i]           # (N,4)

            keep = score > conf_thresh
            score = score[keep]
            label = label[keep]
            box = box[keep]

            # NMS
            if box.shape[0] > 0:
                keep_idx = nms(box, score, iou_threshold=0.5)
                box = box[keep_idx]
                label = label[keep_idx]

            # 只保留 top-1 預測
            if box.shape[0] > 0:
                pred_box = box[0].cpu()
                pred_label = label[0].cpu().item()
            else:
                pred_box = None

            gt_box = targets[i]['boxes'][0].cpu()
            gt_label = targets[i]['labels'][0].cpu().item()
            total += 1

            if pred_box is not None:
                iou = compute_iou(pred_box, gt_box)
                if iou > iou_thresh and pred_label == gt_label:
                    correct += 1

    precision = correct / total
    print(f"mAP@0.5 (top-1 prediction only): {precision:.4f}")
    return precision

In [19]:
# 凍結 blocks.0 和 blocks.1
for name, param in model.backbone.named_parameters():
    if 'blocks.0' in name or 'blocks.1' in name:
        param.requires_grad = False


for name, param in model.named_parameters():
    if 'seg' in name or 'cls' in name:
        param.requires_grad = False

In [20]:
import torch.optim as optim
# 初始化
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

n_det_cls = len(det_cat_mapping)  # 假設 det_cat_mapping: 原始ID → 0開始
n_anchor = 3
anchors = torch.tensor([[1.0, 1.0], [2.0, 2.0], [0.5, 0.5]])  # (A, 2)

#model = UniHeadNet(n_det_cls=n_det_cls, n_anchor=n_anchor).to(device)

# Dataset / Dataloader
train_dataset = COCODetectionDataset(
    img_dir=f'{data_root}/data/mini_coco_det/train',
    ann_path=f'{data_root}/data/mini_coco_det/annotations/instances_train2017.json',
    det_cat_mapping=det_cat_mapping,
    size=img_size
)

val_dataset = COCODetectionDataset(
    img_dir=f'{data_root}/data/mini_coco_det/val',
    ann_path=f'{data_root}/data/mini_coco_det/annotations/instances_val2017.json',
    det_cat_mapping=det_cat_mapping,
    size=img_size
)

train_loader = DataLoader(
    train_dataset, batch_size=16, shuffle=True,
    collate_fn=collate_fn, num_workers=2
)
val_det_loader = DataLoader(
    val_dataset, batch_size=16, shuffle=False,
    collate_fn=collate_fn, num_workers=2
)
optimizer = optim.Adam(model.parameters(), lr=2e-3)


best_mAP=0
for epoch in range(40):
    loss = train_one_epoch(model, train_loader, optimizer,
                           anchors=anchors, num_classes=n_det_cls, device=device)
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}")
    #map50 = evaluate_map(model, train_loader, anchors, num_classes=n_det_cls, device=device)
    map50 = evaluate_map(model, val_det_loader, anchors, num_classes=n_det_cls, device=device)

    if map50 > best_mAP:
      best_mAP = map50
      torch.save(model.state_dict(), 'best_det_model.pth')
      print(f"Best mAP is {best_mAP}")
      print("save model")

100%|██████████| 15/15 [00:04<00:00,  3.58it/s]


Epoch 1, Loss: 164.0579


100%|██████████| 4/4 [00:01<00:00,  3.71it/s]


mAP@0.5 (top-1 prediction only): 0.0000


100%|██████████| 15/15 [00:03<00:00,  3.83it/s]


...

mAP@0.5 (top-1 prediction only): 0.3000


100%|██████████| 15/15 [00:03<00:00,  3.81it/s]


Epoch 40, Loss: 0.7625


100%|██████████| 4/4 [00:00<00:00,  4.88it/s]

mAP@0.5 (top-1 prediction only): 0.3167


In [21]:
model.load_state_dict(torch.load('best_det_model.pth'))
model.eval()
val_mAP=evaluate_map(model, val_det_loader, anchors, num_classes=n_det_cls, device=device)
print(f"mAP_base is:{val_mAP}")


_,val_miou =validate_seg(model, val_seg_loader, cfg["device"])

print(f"mIoU_drop is:{val_miou}")

100%|██████████| 4/4 [00:00<00:00,  4.56it/s]


mAP@0.5 (top-1 prediction only): 0.3167
mAP_base is:0.31666666666666665


100%|██████████| 8/8 [00:02<00:00,  3.98it/s]

Val Loss: 0.5360, mIoU: 0.9301
mIoU_drop is:0.9301023855805397


# cls

In [22]:
from torch.utils.data import Dataset
from torchvision import transforms
from pathlib import Path
from PIL import Image


from torchvision import transforms

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(img_size, scale=(0.5, 1.0)),
    transforms.RandomApply([
        transforms.CenterCrop(224),
        transforms.Resize((img_size, img_size))
    ], p=0.8),  # 80% 機率強制視野遮蔽（模仿 occlusion）

    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.RandomRotation(25),

    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.4, hue=0.1),
    transforms.RandomGrayscale(p=0.1),

    transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 2.0)),
#     transforms.RandomApply([
#         transforms.Lambda(lambda x: TF.adjust_sharpness(x, sharpness_factor=2))
#     ], p=0.3),

    transforms.ToTensor(),

    # Normalize (ImageNet)
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

class ImageClsDataset(Dataset):
    def __init__(self, root, transform):
        self.root = Path(root)
        self.paths = list(self.root.glob("*/*.jpeg"))
        self.transform = transform

    def __getitem__(self, idx):
        img_path = self.paths[idx]
        label = int(img_path.parent.name)
        img = Image.open(img_path).convert("RGB")
        img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.paths)

In [23]:

train_ds = ImageClsDataset(f"{data_root}/data/imagenette_160/train", transform=train_transform)
val_ds   = ImageClsDataset(f"{data_root}/data/imagenette_160/val",   transform=val_transform)

train_dl = DataLoader(train_ds, batch_size=4, shuffle=True,  num_workers=2)
val_dl   = DataLoader(val_ds,   batch_size=16, shuffle=False, num_workers=2)

In [24]:
def train_epoch(model, dataloader, optimizer,scheduler, device):
    model.train()
    total_loss, total_correct, total_samples = 0, 0, 0
    all_preds, all_labels = [], []

    for imgs, labels in tqdm(dataloader, desc="Train"):
        imgs, labels = imgs.to(device), labels.to(device)

        logits = model(imgs)['cls']  # (B, C)
        y = F.one_hot(labels, num_classes=logits.shape[1]).float()

        loss = F.binary_cross_entropy_with_logits(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item() * imgs.size(0)

        # metrics
        probs = torch.sigmoid(logits).detach()
        preds = torch.argmax(probs, dim=1)
        total_correct += (preds == labels).sum().item()
        total_samples += imgs.size(0)

        all_preds.append(probs.cpu())
        all_labels.append(y.cpu())

    # metrics
    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()

    try:
        auc = roc_auc_score(all_labels, all_preds, average='macro', multi_class='ovr')
    except:
        auc = 0.0  # AUC may fail on few classes per batch

    acc = total_correct / total_samples
    avg_loss = total_loss / total_samples
    return avg_loss, acc, auc
@torch.no_grad()
def eval_epoch(model, dataloader, device):
    model.eval()
    total_loss, total_correct, total_samples = 0, 0, 0
    all_preds, all_labels = [], []

    for imgs, labels in tqdm(dataloader, desc="Val"):
        imgs, labels = imgs.to(device), labels.to(device)

        logits = model(imgs)['cls']
        y = F.one_hot(labels, num_classes=logits.shape[1]).float()

        loss = F.binary_cross_entropy_with_logits(logits, y)
        total_loss += loss.item() * imgs.size(0)

        probs = torch.sigmoid(logits)
        preds = torch.argmax(probs, dim=1)
        total_correct += (preds == labels).sum().item()
        total_samples += imgs.size(0)

        all_preds.append(probs.cpu())
        all_labels.append(y.cpu())

    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()

    try:
        auc = roc_auc_score(all_labels, all_preds, average='macro', multi_class='ovr')
    except:
        auc = 0.0

    acc = total_correct / total_samples
    avg_loss = total_loss / total_samples
    return avg_loss, acc, auc

In [25]:
model.load_state_dict(torch.load('best_det_model.pth'))

# 凍結 backbone 全部
for param in model.backbone.parameters():
    param.requires_grad = False

# 凍結 reduce（Conv1x1 + BN + SiLU）
for param in model.reduce.parameters():
    param.requires_grad = False

# 凍結 head_conv
for name, param in model.head_conv.named_parameters():
      param.requires_grad = False

for name, param in model.head_out.named_parameters():
      param.requires_grad = True


# # for param in model.backbone.parameters():
# #     param.requires_grad = False

In [26]:
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-3)

num_epochs = 36
train_steps_per_epoch = len(train_dl)
total_steps = num_epochs * train_steps_per_epoch
warmup_steps = int(0.05 * total_steps)  # 可調整比例

scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

best_top1 = 0

for epoch in range(1, num_epochs+1):
    train_loss, train_acc, train_auc = train_epoch(model, train_dl, optimizer,scheduler, device)
    val_loss, val_acc, val_auc = eval_epoch(model, val_dl, device)

    print(f"[Epoch {epoch}]")
    print(f"  Train | Loss: {train_loss:.4f}  Acc: {train_acc:.4f}  AUC: {train_auc:.4f}")
    print(f"  Val   | Loss: {val_loss:.4f}  Acc: {val_acc:.4f}  AUC: {val_auc:.4f}")

    if val_acc > best_top1:
        best_top1 = val_acc
        torch.save(model.state_dict(), "best_cls_model.pth")
        print("  ✔️  New best AUC model saved.")


Val: 100%|██████████| 4/4 [00:00<00:00,  7.59it/s]


[Epoch 1]
  Train | Loss: 0.3681  Acc: 0.0292  AUC: 0.4977
  Val   | Loss: 0.1489  Acc: 0.0667  AUC: 0.5103
  ✔️  New best AUC model saved.


Val: 100%|██████████| 4/4 [00:00<00:00,  7.87it/s]


[Epoch 2]
  Train | Loss: 0.1512  Acc: 0.0250  AUC: 0.4358
  Val   | Loss: 0.1499  Acc: 0.0667  AUC: 0.5434


Val: 100%|██████████| 4/4 [00:00<00:00,  7.78it/s]


...

[Epoch 34]
  Train | Loss: 0.1346  Acc: 0.1875  AUC: 0.7394
  Val   | Loss: 0.1388  Acc: 0.1167  AUC: 0.7322


Val: 100%|██████████| 4/4 [00:00<00:00,  7.55it/s]


[Epoch 35]
  Train | Loss: 0.1341  Acc: 0.1458  AUC: 0.7455
  Val   | Loss: 0.1390  Acc: 0.1167  AUC: 0.7302


Val: 100%|██████████| 4/4 [00:00<00:00,  4.66it/s]

[Epoch 36]
  Train | Loss: 0.1337  Acc: 0.1958  AUC: 0.7556
  Val   | Loss: 0.1381  Acc: 0.1500  AUC: 0.7239


In [27]:
modelA = UniHeadNet(n_seg_cls=19,n_det_cls=10, n_anchor=3,n_cls=30).to(cfg["device"])
modelB = UniHeadNet(n_seg_cls=19,n_det_cls=10, n_anchor=3,n_cls=30).to(cfg["device"])
modelA.load_state_dict(torch.load('best_det_model.pth'))

modelB.load_state_dict(torch.load('best_cls_model.pth'))
with torch.no_grad():
    C_det = modelA.C_det
    C_seg = modelA.C_seg

    modelB.head_out.weight[:C_det+C_seg].copy_(
        modelA.head_out.weight[:C_det+C_seg])
    modelB.head_out.bias[:C_det+C_seg].copy_(
        modelA.head_out.bias[:C_det+C_seg])




model=modelB


In [28]:
def diff_report(model_old, model_new, atol=1e-6):
    count=0
    for (n1, p1), (n2, p2) in zip(model_old.state_dict().items(),
                                   model_new.state_dict().items()):
        if not torch.allclose(p1, p2, atol=atol):
            count+=1
            print(f"{n1:60s}  changed   max|Δ|={float((p1-p2).abs().max()):.3e}")
        if count>10:
            print("...")
            break
diff_report(modelA, modelB)

backbone.bn1.running_mean                                     changed   max|Δ|=5.157e+00
backbone.bn1.running_var                                      changed   max|Δ|=3.565e+01
backbone.bn1.num_batches_tracked                              changed   max|Δ|=1.140e+03
backbone.blocks.0.0.bn1.running_mean                          changed   max|Δ|=4.520e-01
backbone.blocks.0.0.bn1.running_var                           changed   max|Δ|=8.179e+01
backbone.blocks.0.0.bn1.num_batches_tracked                   changed   max|Δ|=1.140e+03
backbone.blocks.0.0.bn2.running_mean                          changed   max|Δ|=7.846e-02
backbone.blocks.0.0.bn2.running_var                           changed   max|Δ|=1.349e+00
backbone.blocks.0.0.bn2.num_batches_tracked                   changed   max|Δ|=1.140e+03
backbone.blocks.1.0.bn1.running_var                           changed   max|Δ|=1.006e+02
backbone.blocks.1.0.bn1.num_batches_tracked                   changed   max|Δ|=1.140e+03
...


In [29]:
model.eval()
_,top1_score,_ = eval_epoch(model, val_dl, device)
print(f"Top1_base is:{top1_score}")
val_mAP=evaluate_map(model, val_det_loader, anchors, num_classes=n_det_cls, device=device)
print(f"mAP_drop is:{val_mAP}")

_,val_miou =validate_seg(model, val_seg_loader, cfg["device"])

print(f"mIoU_drop is:{val_miou}")

Val: 100%|██████████| 4/4 [00:00<00:00,  6.13it/s]


Top1_base is:0.16666666666666666


100%|██████████| 4/4 [00:00<00:00,  4.18it/s]


mAP@0.5 (top-1 prediction only): 0.1667
mAP_drop is:0.16666666666666666


100%|██████████| 8/8 [00:01<00:00,  4.59it/s]

Val Loss: 0.5364, mIoU: 0.9375
mIoU_drop is:0.9375106692314148


In [30]:
torch.save(model.state_dict(), 'best_model.pt')

In [31]:
time_end=time.time()

if torch.cuda.is_available():
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    print("No GPU available.")

In [32]:
if torch.cuda.is_available(): print("Using GPU:", torch.cuda.get_device_name(0))
else: print("No GPU available.")

Using GPU: Tesla T4


In [33]:
elapsed = time_end-time_start

h, rem = divmod(elapsed, 3600)
m, s = divmod(rem, 60)

print(f"{int(h)}h {int(m)}m {s:.2f}s")

0h 10m 59.83s
